<a href="https://colab.research.google.com/github/ahphanikrishna/TradingModel/blob/main/Hieken_Ashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pkg_resources

required = {'quandl'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
  !pip install quandl

In [ ]:
import os
import time
import pickle
import logging

import quandl

import pandas as pd
import numpy as np
from datetime import datetime

import statsmodels

import statsmodels.api as sm

from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

from statsmodels.tools.eval_measures import rmse, aic

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
%matplotlib inline

print(os.listdir("."))

['.config', 'drive', 'Equity.csv', 'sample_data']


In [ ]:
# Set the standard Variables
QUANDL_KEY = "zSknuWKyh6Cny1ZyvdYi"
quandl.ApiConfig.api_key = QUANDL_KEY

STOCK_PRICES_PATH = "/content/drive/My Drive/AlgoTrading/Stock Prices/"

# Reading the Stocks traded in Bombay Stock exchange
STOCKLISTFILE = "/content/drive/My Drive/AlgoTrading/Equity.csv" 
#STOCKLISTFILE = "/content/Equity.csv"
STOCK_LIST_0 = pd.read_csv(STOCKLISTFILE)

STOCK_LIST_0.shape
STOCK_LIST_0.head()

STOCK_LIST = STOCK_LIST_0[STOCK_LIST_0.Instrument == "Equity"]
# STOCK_LIST = STOCK_LIST_0
STOCK_LIST.shape

(492, 10)

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500003,NaN,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Oil Marketing & Distribution,Equity
1,500008,NaN,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.0,INE885A01032,Auto Parts & Equipment,Equity
2,500010,NaN,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,2.0,INE001A01036,Housing Finance,Equity
3,500020,NaN,BOMDYEING,BOMBAY DYEING & MFG.CO.LTD.,Active,A,2.0,INE032A01023,Textiles,Equity
4,500027,NaN,ATUL,ATUL LTD.,Active,A,10.0,INE100A01010,Specialty Chemicals,Equity


(492, 10)

In [ ]:
def extractAllStockData():
    for index,row in STOCK_LIST.iterrows():
        print(str(datetime.now())," Fetching data for "+row['Security Name'])

        code = str(row['Security Code'])
        quandlStockCode = 'BSE/BOM'+ code
        filename= STOCK_PRICES_PATH + row['Security Id'] + '.csv'

        try:
            StockDataDF = quandl.get(quandlStockCode,start_date="2010-01-01", end_date="2019-12-31")
            print(StockDataDF.head())

            #StockDataDF.to_csv(filename)
        except Exception as e:
            print(str(datetime.now())," Unable to fetch data for "+row['Security Name'])
            print(e)
    
        try:
            StockDataDF.to_csv(filename)
        except Exception as e:
            print(str(datetime.now())," Unable to write data for "+row['Security Name'])
            print(e)

    return

extractAllStockData()

In [ ]:
# Create a full dataframe which will have PCT change for all stocks in BSE
FULL_DF_0 = pd.DataFrame()

# Calculate percentage change for all stocks
for index,row in STOCK_LIST.iterrows():
  code = str(row['Security Code'])
  symbol = row['Security Id']
  FILE = STOCK_PRICES_PATH + row['Security Id'] + '.csv'

  if os.path.exists(FILE):
    # Read the Stock File
    DF = pd.read_csv(FILE)
    DF['Date'] = pd.to_datetime(DF['Date'], format='%Y/%m/%d')

    # Calculate Daily Return on the Closing Price
    DF[symbol] = DF['Close'].pct_change() * 100

    # Fetch only 2019 Data and Pct Return for the stock
    #DF = DF[['Date',symbol]]
    #DF = DF[DF.Date.dt.year == 2019 ]

    # This will initiate the Dataframe for the first time and merge with it in the following runs
    if FULL_DF_0.empty:
      FULL_DF_0 = DF.copy(deep = True)
    else:
      FULL_DF_0 = pd.merge(FULL_DF_0, DF, 
                      on = 'Date',
                      how = 'outer')

FULL_DF_0 = FULL_DF_0.iloc[1:]
FULL_DF_0.set_index('Date',drop = True, inplace = True)
FULL_DF_0.dropna(axis=1, how="any", inplace=True)

FULL_DF_0.shape
FULL_DF_0.head()

HA_OPEN = AVG( T-1 OPEN, T-1 CLOSE)
HA_HIGH = MAX(HA_OPEN, HA_CLOSE, HIGH on T)
HA_LOW = MIN(HA_OPEN, HA_CLOSE, Low on T)
HA_CLOSE = AVG( OPEN, HIGH, LOW, CLOSE of T)
rolling 252 day return
